In [1]:
import sys
import pandas as pd
sys.path.append('/home/wangs/rs/lib')
# sys.path.append('/home/wangs/rs/qza')
import os
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import zipfile
import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import imp
import ff

In [483]:
data = ff.read_binance('BTC')
time_idx = data.index.map(lambda x:datetime.datetime.strptime(x,'%Y%m%d %H%M%S'))
data.index = time_idx

In [2]:
# 所有加密货币的名字
currency_names=['BTC','ETH','XRP','BNB','TRX','ADA','ICP','APE','ARB','ARPA','ATOM','AVAX','BAKE','DOT','ETH',"FIL",'INJ','LINK','MKR','NEAR','NEO','RUNE','SEI','SOL','SUI','UNI','VET','WLD']
# currency_names = ['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'EOS', 'BNB', 'XLM', 'TRX', 'ADA']
# 读取一个币种的高开低收成交量
coin = "BTC"
ff.read_binance(coin)
# 预处理要用到的工具函数
freq = 15 # 15分钟为周期采样
resample_rule = "{}min".format(freq)

# Convert index from str into pd.Datetime
def timing(df):
    df.index = pd.to_datetime(df.index)
    df.index.name = None
    return df

def naming(df, name):
    df.name = name
    return df

def phase_transformer(s): # s = "20220101 010300"
    minute = int(s[11:13])
    return f"{s[:11]}{minute-minute%freq:02d}"

start, end = "20200101", "20240101"
# 读取数据
def currency_data(currency_name):
    global start, end
    data = ff.read_binance(currency_name).rename(columns={'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume'}).loc[start:end]
    data['tradedate'] = data.index.str[:8]
    data['tradehour'] = data.index.str[:11]
    data['tradephase'] = data.index.str[:15]
    data['tradephase'] = data.tradephase.apply(phase_transformer) # 如果要重采样的话，使用函数phase_transformer进行时间切分
    data = timing(data) # 将data的行索引转化为pd.datetime形式
    return data.reindex(pd.date_range(start=pd.to_datetime(start), 
                                      end=pd.to_datetime(end), 
                                      freq="1min", inclusive="left")) # 补充上缺失的时间索引

高频波动因子

In [485]:
data = [currency_data(currency_name)['close'] for currency_name in currency_names]
data = pd.DataFrame(data).T
data.columns = currency_names

In [21]:
ret = data/data.shift(1)
ret_std = ret.rolling(window = 240,closed = 'right').std()
filter_index = ['00:00:00','04:00:00','08:00:00','12:00:00','16:00:00','20:00:00']
ret_std_4 = ret_std[(ret_std).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
ret_std_mean_4 = ret_std_4.rolling(window = 20,closed = 'right').mean()
ret_std_std_4 = ret_std_4.rolling(window = 20,closed = 'right').std()
UID_4 = ret_std_std_4/ret_std_mean_4
UID_4 = UID_4.T
UID_4

,2020-01-01 00:00:00,2020-01-01 04:00:00,2020-01-01 08:00:00,2020-01-01 12:00:00,2020-01-01 16:00:00,2020-01-01 20:00:00,2020-01-02 00:00:00,2020-01-02 04:00:00,2020-01-02 08:00:00,2020-01-02 12:00:00,...,2023-12-30 08:00:00,2023-12-30 12:00:00,2023-12-30 16:00:00,2023-12-30 20:00:00,2023-12-31 00:00:00,2023-12-31 04:00:00,2023-12-31 08:00:00,2023-12-31 12:00:00,2023-12-31 16:00:00,2023-12-31 20:00:00
BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.253523,0.241933,0.256125,0.250689,0.235030,0.233933,0.240006,0.247548,0.259330,0.261723
ETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.306294,0.323744,0.319219,0.305423,0.284250,0.284444,0.285322,0.307709,0.330439,0.321281
XRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.373834,0.313568,0.315847,0.317321,0.328339,0.344115,0.371451,0.395611,0.425273,0.433133
LTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.293563,0.283109,0.280777,0.282478,0.289198,0.313952,0.323211,0.332749,0.360907,0.380796
BCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.407160,0.409630,0.386461,0.368399,0.362686,0.375362,0.396450,0.399010,0.425714,0.442362
EOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.216352,0.195286,0.195512,0.196679,0.204098,0.214072,0.203782,0.207292,0.212147,0.221366
BNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.306250,0.291984,0.314696,0.325815,0.333669,0.358105,0.362299,0.376885,0.379850,0.398074
XLM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.271429,0.243475,0.240490,0.237357,0.274227,0.263716,0.267917,0.276384,0.289960,0.296503
TRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.367215,0.322251,0.308438,0.304256,0.306461,0.306647,0.312535,0.326346,0.335751,0.347295
ADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.408018,0.287442,0.294988,0.301758,0.306130,0.313071,0.333235,0.354120,0.378027,0.379539


In [25]:
ret = data/data.shift(1)
ret_std = ret.rolling(window = 1440,closed = 'right').std()
filter_index = ['00:00:00']
ret_std_24 = ret_std[(ret_std).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
ret_std_mean_24 = ret_std_24.rolling(window = 20,closed = 'right').mean()
ret_std_std_24 = ret_std_24.rolling(window = 20,closed = 'right').std()
UID_24 = ret_std_std_24/ret_std_mean_24
UID_24 = UID_24.T
UID_24

,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,...,2023-12-22,2023-12-23,2023-12-24,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31
BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.307090,0.284847,0.296941,0.326559,0.325911,0.313823,0.314522,0.312306,0.306718,0.280039
ETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.313256,0.295855,0.297117,0.305572,0.308314,0.309780,0.308411,0.314227,0.311126,0.293288
XRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.440588,0.403039,0.410758,0.309207,0.303707,0.303527,0.303457,0.308215,0.291368,0.298178
LTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.317611,0.288739,0.273939,0.280338,0.283787,0.282705,0.297910,0.310657,0.313741,0.313211
BCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.531410,0.502659,0.491173,0.479428,0.490603,0.446459,0.473254,0.527362,0.531544,0.536584
EOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.225599,0.188844,0.162543,0.162009,0.155970,0.157465,0.159637,0.155986,0.155001,0.154057
BNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.283713,0.257502,0.254702,0.260910,0.260828,0.306137,0.350333,0.371318,0.371880,0.361224
XLM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.288278,0.277988,0.276986,0.283095,0.281008,0.281930,0.282142,0.282923,0.282163,0.282792
TRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.347726,0.323766,0.324549,0.323882,0.320857,0.318072,0.317189,0.316710,0.320146,0.321865
ADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.387925,0.340172,0.321228,0.319459,0.304808,0.313265,0.306175,0.306633,0.283765,0.296513


非流动性

In [244]:
data = [currency_data(currency_name)['close'] for currency_name in currency_names]
data = pd.DataFrame(data).T
data.columns = currency_names

In [119]:
ret = data/data.shift(1)
log_ret = np.log(ret)
ret_sum = np.exp(log_ret.rolling(window = 240,closed = 'right').sum())-1

In [120]:
volume = [currency_data(currency_name)['volume'] for currency_name in currency_names]
volume = pd.DataFrame(volume).T
volume.columns = currency_names
volume_data = volume * data / 1000000

In [121]:
volume_data_sum = volume_data.rolling(window = 240,closed = 'right').sum()

In [122]:
ILLIQ = np.abs(ret_sum)/volume_data_sum
filter_index = ['00:00:00','04:00:00','08:00:00','12:00:00','16:00:00','20:00:00']
ILLIQ_mean_4 = ILLIQ[(ILLIQ).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
ILLIQ_mean_4 = ILLIQ_mean_4.rolling(window = 20,closed = 'right').mean()
ILLIQ_mean_4 = np.log(ILLIQ_mean_4+1)

In [123]:
ILLIQ_mean_4

,BTC,ETH,XRP,LTC,BCH,EOS,BNB,XLM,TRX,ADA
2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-12-31 04:00:00,0.000020,0.000050,0.000229,0.001062,0.001681,0.005272,0.000168,0.004439,0.000714,0.000638
2023-12-31 08:00:00,0.000022,0.000050,0.000246,0.001097,0.001747,0.005141,0.000170,0.004398,0.000699,0.000677
2023-12-31 12:00:00,0.000021,0.000053,0.000238,0.001131,0.001842,0.005164,0.000183,0.004636,0.000675,0.000654
2023-12-31 16:00:00,0.000023,0.000054,0.000252,0.001218,0.001768,0.005706,0.000167,0.004676,0.000718,0.000707


一天1440的处理方式完全相同

温和动量

In [298]:
data = [currency_data(currency_name)['close'] for currency_name in currency_names]
data = pd.DataFrame(data).T
data.columns = currency_names

In [342]:
volume = [currency_data(currency_name)['volume'] for currency_name in currency_names]
volume = pd.DataFrame(volume).T
volume.columns = currency_names
volume_data = volume * data
vwap = volume_data.rolling(window = 480,closed = 'right').sum() / volume.rolling(window = 480,closed = 'right').sum()

In [375]:
pre_close = data.shift(481)
filter_index = ['08:00:00']
pre_close = pre_close[(pre_close).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
morning_vwap = vwap[(vwap).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
morning = morning_vwap/pre_close-1
morning_ret = (abs(morning) > 0.02).replace({False:1,True:-1}) * morning

In [339]:
factor_morning_ret_filter = morning.rolling(window = 20,closed = 'left').sum()
factor_morning_ret_filter.index = factor_morning_ret_filter.index.strftime('%Y-%m-%d')

In [362]:
filter_index = ['16:00:00']
noon_vwap = vwap[(vwap).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
noon_ret = (noon_vwap.values - morning_vwap.values)/pre_close

In [383]:
filter_index = ['23:59:00']
close = data[(data).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
tail_ret = (close - noon_vwap.values)/pre_close.values

接下来就是各种三因子和rolling，和之前的数据一样就不仔细计算了

In [391]:
factor_morning_close = (-tail_ret).rolling(window = 5,closed = 'left').sum() + (-noon_ret).rolling(window = 20,closed = 'left').sum().values

In [392]:
factor_mom_spring = factor_morning_close + morning_ret.rolling(window = 20,closed = 'left').sum().values

In [395]:
factor_mom_spring

,BTC,ETH,XRP,LTC,BCH,EOS,BNB,XLM,TRX,ADA
2020-01-01 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-12-27 23:59:00,0.002894,0.013914,0.055114,-0.004367,0.078788,-0.038420,-0.117861,-0.011076,-0.012047,-0.174197
2023-12-28 23:59:00,-0.030082,-0.029504,0.027057,-0.032808,-0.039063,-0.092756,-0.167370,-0.057440,-0.041393,-0.204729
2023-12-29 23:59:00,0.005981,-0.024503,0.038809,-0.037314,-0.086160,-0.073106,-0.189722,-0.032608,-0.014745,-0.146338
2023-12-30 23:59:00,0.005576,-0.017875,0.071098,0.006864,-0.055723,-0.031879,-0.126631,0.002594,-0.012928,-0.044602


高低放量

In [3]:
data = [currency_data(currency_name)['close'] for currency_name in currency_names]
data = pd.DataFrame(data).T
data.columns = currency_names
ret = data/data.shift(1)
log_ret = np.log(ret)
ret_sum = np.exp(log_ret.rolling(window = 240,closed = 'right').sum())-1
filter_index = ['00:00:00','04:00:00','08:00:00','12:00:00','16:00:00','20:00:00']
ret_sum = ret_sum[(ret_sum).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]

In [451]:
ret_judge = (ret_sum > ret_sum.rolling(window = 120).mean() + 2 * ret_sum.rolling(window = 120).std())

In [452]:
close_judge = (data < data.rolling(window = 120).quantile(0.1))

In [123]:
list = np.array([20,2,3,4,15,6,17,8,9,10,11,12,13,14,5,16,7,18,19,1])
arr = np.arange(20)
for i in range(5):
    arr[np.argsort(arr)[4*i:4*i+4]] = i
5*((arr==4) * list).sum()/list.sum()

1.0714285714285714

In [4]:
data = [currency_data(currency_name)['close'] for currency_name in currency_names]
data = pd.DataFrame(data).T
data.columns = currency_names
ret = data/data.shift(1)-1

In [5]:
ret_5 = ret.rolling(window = 5,closed = 'left').std() 

In [20]:
num = (data.shape[0]-4800)/240
para = [4800+240*i for i in range(int(num+1))]
def high_min_ret(n):
    close_data_20 = (data.iloc[n-4800:n,:]>=data.iloc[n-4800:n,:].quantile(0.8)).replace({False:np.nan})
    return ((close_data_20 * ret_5.iloc[n-4800:n,:]).mean() / ret_5.iloc[n-4800:n,:].mean()).values

In [21]:
with Pool(48) as p:
    res_lst = list(tqdm(p.imap(high_min_ret, para), total=len(para)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8747/8747 [00:09<00:00, 906.75it/s]


In [22]:
ret_factor_high = pd.DataFrame(res_lst)

In [23]:
filter_index = ['00:00:00','04:00:00','08:00:00','12:00:00','16:00:00','20:00:00']
ret_sum = data[(data).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]

In [28]:
ret_factor_high.columns = data.columns
ret_factor_high.index = ret_sum.index[18:-1]

In [29]:
ret_factor_high

,BTC,ETH,XRP,BNB,TRX,ADA,ICP,APE,ARB,ARPA,...,MKR,NEAR,NEO,RUNE,SEI,SOL,SUI,UNI,VET,WLD
2020-01-04 00:00:00,1.480866,1.290914,0.713859,0.809787,1.107404,1.332888,NaN,NaN,NaN,0.941828,...,NaN,NaN,1.170210,NaN,NaN,NaN,NaN,NaN,0.990929,NaN
2020-01-04 04:00:00,1.347329,1.259410,0.727480,0.749701,1.108392,1.202092,NaN,NaN,NaN,0.991803,...,NaN,NaN,1.208370,NaN,NaN,NaN,NaN,NaN,0.997508,NaN
2020-01-04 08:00:00,1.229086,1.183459,0.726493,0.749810,1.051576,1.133027,NaN,NaN,NaN,1.046310,...,NaN,NaN,1.138293,NaN,NaN,NaN,NaN,NaN,1.002276,NaN
2020-01-04 12:00:00,1.139380,1.025993,0.708518,0.763291,1.017558,1.134335,NaN,NaN,NaN,1.009727,...,NaN,NaN,1.085169,NaN,NaN,NaN,NaN,NaN,1.006911,NaN
2020-01-04 16:00:00,1.107342,0.998635,0.792350,0.804582,1.015752,1.128208,NaN,NaN,NaN,0.921380,...,NaN,NaN,1.070326,NaN,NaN,NaN,NaN,NaN,1.008049,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 00:00:00,0.948095,1.239000,1.279443,1.469268,0.954756,1.391292,1.407624,1.124192,1.341486,1.121504,...,1.234308,1.110899,1.020045,1.079717,1.180278,1.047406,1.109059,1.223461,1.125929,1.350522
2023-12-31 04:00:00,0.986617,1.272785,1.362839,1.492001,0.950154,1.426943,1.446441,1.104854,1.348312,1.142013,...,1.216641,1.144771,1.071562,1.038875,1.069861,1.044952,1.131896,1.238794,1.093937,1.376562
2023-12-31 08:00:00,0.976986,1.287298,1.457986,1.489875,0.968874,1.527116,1.360484,1.110792,1.358775,1.203599,...,1.211964,1.226356,1.151928,1.050352,0.973291,1.056607,1.236318,1.244896,1.109766,1.350202
2023-12-31 12:00:00,1.016139,1.246553,1.361003,1.515720,0.974494,1.450092,1.372909,1.041226,1.247633,1.193211,...,1.227195,1.229201,1.145167,1.048765,0.885796,1.064138,1.205787,1.272384,1.092645,1.343930
